# Project in AI in Industry: Predicting Road Accident Risk

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from util.data import load_data, clean_data, modify_features

### Important constants

In [ ]:
TRAIN_DATA = "data/train.csv"
TEST_DATA  = "data/test.csv"

### Checking out data

In [ ]:
df_train = load_data(TRAIN_DATA)

df_test = load_data(TRAIN_DATA)